In [1]:
import pandas as pd

from lets_plot import *
from lets_plot import tilesets
from lets_plot.geo_data import *

The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL).


In [2]:
LetsPlot.setup_html()

In [3]:
plot_side = 600
colors = dict(
    light=dict(
        color1="#d9d9d9",
        color2="#f7f4f0",
        color3="#c9c9c9",
    ),
    dark=dict(
        color1="#141414",
        color2="#3d3d3d",
        color3="#363636",
    ),
)

## Charts

In [4]:
charts_df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/penguins.csv")
charts_df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female


In [5]:
def charts_plot(theme_type):
    if theme_type == 'dark':
        line_color = "white"
    else:
        line_color = "#474747"
    return ggplot(charts_df) + \
        geom_ydotplot(aes("species", "body_mass_g"), dotsize=.5, stackratio=1.2, stroke=0, \
                      fill="#9a5353", tooltips='none') + \
        scale_y_continuous(name="body mass (g)") + \
        ggsize(plot_side, plot_side) + \
        theme(plot_background=element_rect(fill=colors[theme_type]["color1"]), \
              panel_grid_major=element_line(color=colors[theme_type]["color3"]), \
              axis_line=element_line(color=line_color), \
              axis_ticks=element_line(color=line_color), \
              text=element_text(color=line_color))

In [6]:
charts_plot('light')

In [7]:
charts_plot('dark')

## Maps

In [8]:
maps_state_gdf = geocode_states("California").inc_res(2).get_boundaries()
maps_state_gdf

,state,found name,geometry
0,California,California,"MULTIPOLYGON (((-124.32694 40.6162, -124.31188..."


In [9]:
def get_maps_county_data():
    states = ["California", "Nevada", "Oregon", "Idaho", "Utah", "Arizona"]
    return pd.concat([geocode_counties().scope(state).inc_res(2).get_boundaries() for state in states])

maps_counties_gdf = get_maps_county_data()
maps_counties_gdf.head()

,county,found name,geometry
0,Humboldt County,Humboldt County,"MULTIPOLYGON (((-124.12151 40.97252, -124.1129..."
1,Tulare County,Tulare County,"MULTIPOLYGON (((-119.57338 36.48875, -119.4663..."
2,Sacramento County,Sacramento County,"MULTIPOLYGON (((-121.70842 38.09103, -121.7079..."
3,Marin County,Marin County,"MULTIPOLYGON (((-122.70971 37.8959, -122.82343..."
4,Contra Costa County,Contra Costa County,"MULTIPOLYGON (((-122.31363 37.89706, -122.3907..."


In [10]:
def get_maps_data():
    from sklearn.datasets import fetch_california_housing
    california_housing_bunch = fetch_california_housing()
    df = pd.DataFrame(california_housing_bunch.data, columns=california_housing_bunch.feature_names)
    df["Value"] = california_housing_bunch.target
    return df.sample(n=1000, random_state=42).reset_index(drop=True)

maps_df = get_maps_data()
maps_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Value
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01,0.47700
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46,0.45800
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44,5.00001
3,5.7376,17.0,6.163636,1.020202,1705.0,3.444444,34.28,-118.72,2.18600
4,3.7250,34.0,5.492991,1.028037,1063.0,2.483645,36.62,-121.93,2.78000


In [11]:
def maps_plot(theme_type):
    state_border_color = "#7c334f"
    if theme_type == "dark":
        tiles = tilesets.LETS_PLOT_DARK
        state_alpha_color = "white"
        state_alpha_value = .1
    else:
        tiles = tilesets.LETS_PLOT_LIGHT
        state_alpha_color = "white"
        state_alpha_value = .5
    return ggplot() + \
        geom_livemap(tiles=tiles, location=[-119.3, 37.4], zoom=6) + \
        geom_polygon(data=maps_state_gdf, fill=state_alpha_color, color=state_border_color, \
                     alpha=state_alpha_value, size=.7) + \
        geom_polygon(data=maps_counties_gdf, color=colors[theme_type]["color3"], alpha=0, size=.1) + \
        geom_point(aes(x="Longitude", y="Latitude", size="Value", color="HouseAge"), \
                   data=maps_df, alpha=.8, tooltips='none') + \
        scale_color_viridis(option='twilight', direction=-1, guide='none') + \
        scale_size(range=[5, 15], guide='none') + \
        ggsize(plot_side, plot_side) + \
        theme_void() + theme(plot_inset=0)

In [12]:
maps_plot('light')

In [13]:
maps_plot('dark')

## Geocoding

In [14]:
country = "Ireland"
delta = 4
ratio = 1.7
xmin = -10.25 - (ratio - 1) * delta / 2
xmax = xmin + ratio * delta
ymin = 51.4
ymax = ymin + delta

In [15]:
geocoding_country_gdf = geocode_countries(names=country).inc_res(4).get_boundaries()
geocoding_country_gdf

,country,found name,geometry
0,Ireland,Ireland,"MULTIPOLYGON (((-10.65036 52.07708, -10.65695 ..."


In [16]:
geocoding_counties_gdf = geocode_counties().scope(country).get_boundaries()
geocoding_counties_gdf.head()

,county,found name,geometry
0,County Wicklow,County Wicklow,"MULTIPOLYGON (((-6.73725 52.91578, -6.77737 53..."
1,County Dublin,County Dublin,"MULTIPOLYGON (((-6.47524 53.23431, -6.46013 53..."
2,County Meath,County Meath,"MULTIPOLYGON (((-7.34362 53.79906, -7.00216 53..."
3,County Waterford,County Waterford,"MULTIPOLYGON (((-8.10477 52.20645, -7.37209 52..."
4,County Monaghan,County Monaghan,"MULTIPOLYGON (((-7.31991 54.11347, -7.33956 54..."


In [17]:
def geocoding_plot(theme_type):
    return ggplot() + \
        geom_map(data=geocoding_country_gdf, fill=colors[theme_type]["color2"], size=0) + \
        geom_rect(data=geocoding_counties_gdf, color="#9a5353", size=2, alpha=0) + \
        geom_rect(data=geocoding_counties_gdf, color=colors[theme_type]["color1"], size=1, alpha=0) + \
        coord_fixed(ratio=ratio, xlim=[xmin, xmax], ylim=[ymin, ymax]) + \
        ggsize(plot_side, plot_side) + \
        theme_void() + theme(plot_background=element_rect(fill=colors[theme_type]["color1"]))

In [18]:
geocoding_plot('light')

In [19]:
geocoding_plot('dark')